## Description
1) Segment Hoechst channel to define nuclei

2) Track segmented nuclei using bayesian tracker

3) Export segmented and tracked nuclei

In [63]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import napari
import os
import pickle
import image_utility_functions as iuf
import tifffile as tf
from tqdm.notebook import tqdm
import btrack
from skimage.segmentation import clear_border
from stardist.plot import render_label
from csbdeep.utils import normalize
from stardist.data import test_image_nuclei_2d
from stardist.models import StarDist2D 

In [2]:
# I just need to import the data dictionary with all the info for this notebook
with open('D:/Documents/_phd/main/lab/p01_jfdyes_snap_halo_sensor_fret_response/experiments/e024_20191122/analysis/analysis6/scripts/image_paths.pickle', 'rb') as handle:
    data_dict = pickle.load(handle)
print('Data loaded!')

# export path definition
output_path = "D:/Documents/_phd/main/lab/p01_jfdyes_snap_halo_sensor_fret_response/experiments/e024_20191122/analysis/analysis7/output/"

Data loaded!


In [3]:
# prints a list of available models 
StarDist2D.from_pretrained() 

# creates a pretrained model
model = StarDist2D.from_pretrained('2D_versatile_fluo')

There are 4 registered models for 'StarDist2D':

Name                  Alias(es)
────                  ─────────
'2D_versatile_fluo'   'Versatile (fluorescent nuclei)'
'2D_versatile_he'     'Versatile (H&E nuclei)'
'2D_paper_dsb2018'    'DSB 2018 (from StarDist 2D paper)'
'2D_demo'             None
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [16]:
# Open all Hoechst channel images to check quality
## !! AKAR nuclear images are not usable -> see 1.1_akar_nuclei_cytoplasm_detection.ipynb notebook for independent segmentation !! ##
fluorophores = data_dict.keys()
viewer = napari.Viewer()
for fluo in fluorophores:
    img_path = data_dict[fluo]['xy1']['int_img_path']['ch5']
    nuc_img = tf.imread(img_path)
    viewer.add_image(nuc_img, name=fluo)

divide by zero encountered in true_divide


In [74]:
fluorophores = [fluo for fluo in data_dict.keys() if 'ecfp_ypet' not in fluo]

for fluo in fluorophores:
    folder = data_dict[fluo]['xy1']['int_img_path']['ch5'][:data_dict[fluo]['xy1']['int_img_path']['ch5'].find('/', -20)]

    file_list = [folder+'/'+file for file in os.listdir(folder) if 'c5' in file]
    nuclei = tf.imread(file_list)
    print('Loaded nuclei image for {}'.format(fluo))
    label_list = []
    
    print('Segmenting image..')
    for img in tqdm(nuclei):
        labels, _ = model.predict_instances(normalize(img))
        label_list.append(labels)
        
    nuclei_labels = np.array(label_list)
    objects = btrack.utils.segmentation_to_objects(nuclei_labels)
    # Tracks defined objects across time
    with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure_from_file('cell_config.json')

        # append the objects to be tracked
        tracker.append(objects) 
        # set the volume (Z axis volume is set very large for 2D data)
        tracker.volume=((0, 2000), (0, 2000), (-1e5, 1e5))

        # track them (in interactive mode)
        tracker.track_interactive(step_size=100)

        # generate hypotheses and run the global optimizer
        tracker.optimize()

        # get the tracks as a python list
        tracks = tracker.tracks

        # get the data in a format for napari
        data, properties, graph = tracker.to_napari(ndim=2)

    # # Use tracks from btrack to create masks with same label
    print("Converting trackes to mask..")
    tracked_masks = np.array(iuf.TracksToMasks(data, nuclei_labels))
    
    # Remove nuclei touching the image border
    print("Filtering unwanted nuclei..")
    cleared_mask = tracked_masks.copy()
    for layID, layer in enumerate(tracked_masks):
        cleared_mask[layID] = clear_border(layer)

    # Create an array Layers X Labels that has True if label is present or False if not
    layers_x_labels = []
    for layer in cleared_mask:
        layers_x_labels.append(iuf.select_labels(layer, cleared_mask.max()))
    final = np.array(layers_x_labels)

    # check if a give label is True in all time points
    # filter all possible available labels by those present in all time points
    common_labels = np.arange(0, cleared_mask.max()+1)[final.sum(axis=0) == 20]
    # create final mask with those common labels
    final_mask = np.isin(cleared_mask, common_labels) * cleared_mask
    print("Exporting tracked nuclei..")
    tf.imwrite(output_path + "tracked_nuclei/" + fluo +"_xy1_tracked_nuclei.tif", np.uint16(final_mask))

Loaded nuclei image for jf503snap_jf549halo
Segmenting image..


  0%|          | 0/20 [00:00<?, ?it/s]

[INFO][2022/01/31 10:37:39 AM] Localizing objects from segmentation...
[INFO][2022/01/31 10:37:40 AM] Objects are of type: <class 'dict'>
[INFO][2022/01/31 10:37:41 AM] ...Found 11723 objects in 20 frames.
[INFO][2022/01/31 10:37:41 AM] Loaded btrack: c:\users\marco dalla vecchia\appdata\local\programs\python\python37\lib\site-packages\btrack\libs\libtracker.DLL
[INFO][2022/01/31 10:37:41 AM] btrack (v0.4.2) library imported
[INFO][2022/01/31 10:37:41 AM] Setting max XYZ search radius to: 100
[INFO][2022/01/31 10:37:41 AM] Starting BayesianTracker session
[INFO][2022/01/31 10:37:41 AM] Loading configuration file: cell_config.json
[INFO][2022/01/31 10:37:41 AM] Loading motion model: b'cell_motion'
[INFO][2022/01/31 10:37:41 AM] Objects are of type: <class 'list'>
[INFO][2022/01/31 10:37:41 AM] Set volume to ((0, 2000), (0, 2000), (-100000.0, 100000.0))
[INFO][2022/01/31 10:37:41 AM] Starting tracking... 
[INFO][2022/01/31 10:37:41 AM] Tracking objects in frames 0 to 20 (of 20)...
[INFO]

Converting trackes to mask..
Filtering unwanted nuclei..
Exporting tracked nuclei..
Loaded nuclei image for jf503snap_jf585halo
Segmenting image..


  0%|          | 0/20 [00:00<?, ?it/s]

...

[INFO][2022/01/31 10:53:10 AM] Localizing objects from segmentation...
[INFO][2022/01/31 10:53:12 AM] Objects are of type: <class 'dict'>
[INFO][2022/01/31 10:53:12 AM] ...Found 22512 objects in 20 frames.
[INFO][2022/01/31 10:53:12 AM] Loaded btrack: c:\users\marco dalla vecchia\appdata\local\programs\python\python37\lib\site-packages\btrack\libs\libtracker.DLL
[INFO][2022/01/31 10:53:12 AM] btrack (v0.4.2) library imported
[INFO][2022/01/31 10:53:12 AM] Setting max XYZ search radius to: 100
[INFO][2022/01/31 10:53:12 AM] Starting BayesianTracker session
[INFO][2022/01/31 10:53:12 AM] Loading configuration file: cell_config.json
[INFO][2022/01/31 10:53:12 AM] Loading motion model: b'cell_motion'
[INFO][2022/01/31 10:53:12 AM] Objects are of type: <class 'list'>
[INFO][2022/01/31 10:53:12 AM] Set volume to ((0, 2000), (0, 2000), (-100000.0, 100000.0))
[INFO][2022/01/31 10:53:12 AM] Starting tracking... 
[INFO][2022/01/31 10:53:13 AM] Tracking objects in frames 0 to 20 (of 20)...
[INFO]

Converting trackes to mask..
Filtering unwanted nuclei..
Exporting tracked nuclei..
Loaded nuclei image for jf549halo_jf635snap
Segmenting image..


  0%|          | 0/20 [00:00<?, ?it/s]

[INFO][2022/01/31 10:54:47 AM] Localizing objects from segmentation...
[INFO][2022/01/31 10:54:49 AM] Objects are of type: <class 'dict'>
[INFO][2022/01/31 10:54:50 AM] ...Found 21784 objects in 20 frames.
[INFO][2022/01/31 10:54:50 AM] Loaded btrack: c:\users\marco dalla vecchia\appdata\local\programs\python\python37\lib\site-packages\btrack\libs\libtracker.DLL
[INFO][2022/01/31 10:54:50 AM] btrack (v0.4.2) library imported
[INFO][2022/01/31 10:54:50 AM] Setting max XYZ search radius to: 100
[INFO][2022/01/31 10:54:50 AM] Starting BayesianTracker session
[INFO][2022/01/31 10:54:50 AM] Loading configuration file: cell_config.json
[INFO][2022/01/31 10:54:50 AM] Loading motion model: b'cell_motion'
[INFO][2022/01/31 10:54:50 AM] Objects are of type: <class 'list'>
[INFO][2022/01/31 10:54:50 AM] Set volume to ((0, 2000), (0, 2000), (-100000.0, 100000.0))
[INFO][2022/01/31 10:54:50 AM] Starting tracking... 
[INFO][2022/01/31 10:54:51 AM] Tracking objects in frames 0 to 20 (of 20)...
[INFO]

Converting trackes to mask..
Filtering unwanted nuclei..
Exporting tracked nuclei..


## Below testing code

In [60]:


label_list = []
polygon_list = []

for img in tqdm(nuclei):
    labels, polygons = model.predict_instances(normalize(img))
    
    label_list.append(labels)
    polygon_list.append(polygons)

  0%|          | 0/20 [00:00<?, ?it/s]

In [61]:
nuclei_labels = np.array(label_list)

viewer = napari.Viewer()
viewer.add_image(nuclei)
viewer.add_labels(nuclei_labels)

divide by zero encountered in true_divide


<Labels layer 'nuclei_labels' at 0x1caaa1c2748>

In [7]:
objects = btrack.utils.segmentation_to_objects(nuclei_labels)

# Tracks defined objects across time
with btrack.BayesianTracker() as tracker:
    # configure the tracker using a config file
    tracker.configure_from_file('cell_config.json')

    # append the objects to be tracked
    tracker.append(objects) 
    # set the volume (Z axis volume is set very large for 2D data)
    tracker.volume=((0, 2000), (0, 2000), (-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    # get the tracks as a python list
    tracks = tracker.tracks

    # get the data in a format for napari
    data, properties, graph = tracker.to_napari(ndim=2)

# # Use tracks from btrack to create masks with same label
tracked_masks = np.array(iuf.TracksToMasks(data, nuclei_labels))

[INFO][2022/01/20 11:46:52 AM] Localizing objects from segmentation...
[INFO][2022/01/20 11:46:54 AM] Objects are of type: <class 'dict'>
[INFO][2022/01/20 11:46:54 AM] ...Found 16436 objects in 20 frames.
[INFO][2022/01/20 11:46:54 AM] Loaded btrack: c:\users\marco dalla vecchia\appdata\local\programs\python\python37\lib\site-packages\btrack\libs\libtracker.DLL
[INFO][2022/01/20 11:46:54 AM] btrack (v0.4.2) library imported
[INFO][2022/01/20 11:46:54 AM] Setting max XYZ search radius to: 100
[INFO][2022/01/20 11:46:54 AM] Starting BayesianTracker session
[INFO][2022/01/20 11:46:54 AM] Loading configuration file: cell_config.json
[INFO][2022/01/20 11:46:54 AM] Loading motion model: b'cell_motion'
[INFO][2022/01/20 11:46:54 AM] Objects are of type: <class 'list'>
[INFO][2022/01/20 11:46:54 AM] Set volume to ((0, 2000), (0, 2000), (-100000.0, 100000.0))
[INFO][2022/01/20 11:46:54 AM] Starting tracking... 
[INFO][2022/01/20 11:46:54 AM] Tracking objects in frames 0 to 20 (of 20)...
[INFO]

In [9]:
# Remove nuclei touching the image border
cleared_mask = tracked_masks.copy()
for layID, layer in enumerate(tracked_masks):
    cleared_mask[layID] = clear_border(layer)
    
# Create an array Layers X Labels that has True if label is present or False if not
layers_x_labels = []
for layer in cleared_mask:
    layers_x_labels.append(iuf.select_labels(layer, cleared_mask.max()))
final = np.array(layers_x_labels)

# check if a give label is True in all time points
# filter all possible available labels by those present in all time points
common_labels = np.arange(0, cleared_mask.max()+1)[final.sum(axis=0) == 20]
# create final mask with those common labels
final_mask = np.isin(cleared_mask, common_labels) * cleared_mask

In [62]:
tf.imwrite(output_path + "tracked_nuclei/" + fluorophores +"_tracked_nuclei.tif", np.uint16(final_mask))

In [19]:
viewer = napari.Viewer()
viewer.add_labels(final_mask, opacity=0.5)
viewer.add_labels(cells, opacity=0.5)
# viewer.add_tracks(data, properties=properties, graph=graph, name='tracks')

divide by zero encountered in true_divide


<Labels layer 'cells' at 0x219c1f7aec8>